This script generates final reports for external use.

**MAKE SURE THAT ALL LONI SPREADSHEETS ARE DOWNLOADED/UPDATED BEFORE RUNNING THIS SCRIPT**

---------------------------- for data sharing with other sites ------------------------
EXTERNAL REPORT CAN BE FOUND AT: /autofs/cluster/animal/scan_data/leads/spreadsheets/EXTERNAL

------------------------------- for redcap importing ----------------------------------
INTERNAL REPORT CAN BE FOUND AT: /autofs/cluster/animal/scan_data/leads/spreadsheets/INTERNAL

In [1]:
import pandas as pd
import os
import re
from functools import reduce
import time

scandir = '/autofs/cluster/animal/scan_data/leads/recon_nip/SCAN_NOTES/'
internalpath = '/autofs/cluster/animal/scan_data/leads/spreadsheets/INTERNAL/'
externalpath = '/autofs/cluster/animal/scan_data/leads/spreadsheets/EXTERNAL/'
internal_dict = internalpath+'REDCAP_INSTRUMENT/LEADS_DataDictionary_2019-05-14.csv'
external_dict = pd.read_csv(externalpath+'MGH_LEADS_datadictionary_3T_20190422.csv')
file_derivelist = pd.read_csv('/autofs/homes/002/rje11/Desktop/LEADS_QC_Cumulative_20190701_1529.csv')
# treat AMY STATUS as a static variable (scan dates will not match up, and no evidence of scr/baseline for MR tracking)
AMYSTATUS_df = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_PETINFO/LEADS_AMYELG.csv')
DOWNLOADS_df = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv')
RECON3T_QC = pd.read_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_3T/recon_notes.csv')

# stats
statsdir = '/autofs/cluster/animal/scan_data/leads/analyses_nip/RECON_3T/GROUPSTATS/'
stats_asegmean = pd.read_table(statsdir+'leads_aseg_mean_auto.csv', sep=',',index_col=0)
stats_asegvol = pd.read_table(statsdir+'leads_aseg_volume_auto.csv', sep=',',index_col=0)
stats_aseg_std = pd.read_table(statsdir+'leads_aseg_std_auto.csv', sep=',',index_col=0)

# rename col headers for aseg (too ambiguous)
stats_asegmean.columns = [str(col) + '_aseg_mean' for col in stats_asegmean.columns]
stats_asegvol.columns = [str(col) + '_aseg_volume' for col in stats_asegvol.columns]
stats_aseg_std.columns = [str(col) + '_aseg_std' for col in stats_aseg_std.columns]

stats_rh_aparc_vol = pd.read_table(statsdir+'leads_rh_aparc_volume_auto.csv', sep=',',index_col=0)
stats_rh_aparc_area = pd.read_table(statsdir+'leads_rh_aparc_area_auto.csv', sep=',',index_col=0)
stats_rh_aparc_thickness = pd.read_table(statsdir+'leads_rh_aparc_thickness_auto.csv', sep=',',index_col=0)
stats_rh_aparc_thicknessstd = pd.read_table(statsdir+'leads_rh_aparc_thicknessstd_auto.csv', sep=',',index_col=0)
stats_lh_aparc_vol = pd.read_table(statsdir+'leads_lh_aparc_volume_auto.csv', sep=',',index_col=0)
stats_lh_aparc_area = pd.read_table(statsdir+'leads_lh_aparc_area_auto.csv', sep=',',index_col=0)
stats_lh_aparc_thickness = pd.read_table(statsdir+'leads_lh_aparc_thickness_auto.csv', sep=',',index_col=0)
stats_lh_aparc_thicknessstd = pd.read_table(statsdir+'leads_lh_aparc_thicknessstd_auto.csv', sep=',',index_col=0)

stats_rh_aparc_vol.columns = [str(col) + '_aparc' for col in stats_rh_aparc_vol.columns]
stats_lh_aparc_vol.columns = [str(col) + '_aparc' for col in stats_lh_aparc_vol.columns]
stats_rh_aparc_area.columns = [str(col) + '_aparc' for col in stats_rh_aparc_area.columns]
stats_lh_aparc_area.columns = [str(col) + '_aparc' for col in stats_lh_aparc_area.columns]
stats_rh_aparc_thickness.columns = [str(col) + '_aparc' for col in stats_rh_aparc_thickness.columns]
stats_lh_aparc_thickness.columns = [str(col) + '_aparc' for col in stats_lh_aparc_thickness.columns]
stats_rh_aparc_thicknessstd.columns = [str(col) + '_aparc' for col in stats_rh_aparc_thicknessstd.columns]
stats_lh_aparc_thicknessstd.columns = [str(col) + '_aparc' for col in stats_lh_aparc_thicknessstd.columns]

# merge all of the stats data into one dataframe (based on first column)
allstats_df = [stats_asegmean, stats_asegvol, stats_aseg_std, stats_rh_aparc_vol, stats_rh_aparc_area, stats_rh_aparc_thickness, stats_rh_aparc_thicknessstd, stats_lh_aparc_vol, stats_lh_aparc_area, stats_lh_aparc_thickness, stats_lh_aparc_thicknessstd]
statsmerged_df = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), allstats_df).fillna('void')

In [2]:
# go into each folder in /autofs/cluster/animal/scan_data/leads/recon_nip/SCAN_NOTES/ and add scannotes.csv to scaninfo (run, sequence name, dicom status,loni_series,loni_image)
scannotes = pd.DataFrame(columns=['leads_id','mayo_notes','mayo_overallpass','download_date'])
scanlist=[]
sublist=[]
runlist=[]
errorlist=[]
loniimgs = []
loniseries = []
# make a list of all dataframes then concatenate them after the for loop axis =1 
directorylist = os.listdir(scandir)
directorylist.sort()
for file in directorylist:
    if file.startswith("LDS"):
        sublist.append(file.split("_")[0].lower())
        df = pd.read_csv(scandir+file+'/scannotes.csv')
        df2 = pd.read_csv(scandir+file+'/scaninfo.csv',header=None,names=['run','sequence_name','dicom_status','dim1','dim2','dim3','dim4','dicom_name'])
        
        # horizontal concatenation (a subject per loop iteration)
        newdf = df[['leads_id','mayo_notes','mayo_overallpass','download_date']]
        scanlist.append(newdf)
        
        # separate lists that will be appended as columns to final df (vertical concatenation); adds subject's run to list per iteration
        runlist.append(df2['run'].values[0]) 
        errorlist.append(df2['dicom_status'].values[0])
        
        seqname = df2['dicom_name'].values[0]
        loniimgs.append(re.findall(r"_I(\d{0,9}).dcm",seqname)[0])
        loniseries.append(re.findall(r"_S(\d{0,9})_",seqname)[0])
        
allscans = pd.concat(scanlist, axis=0,ignore_index=True)
allscans = allscans.sort_values(by=['leads_id'])
allscans.reset_index(drop=True)

# add runlist, errorlist, seqname, loniimgs, loniseries (vertical concat)
allscans['run_number'] = runlist
allscans['dicom_status'] = errorlist
allscans['loni_image'] = loniimgs
allscans['loni_series'] = loniseries
allscans['subject_code'] = sublist

#just extract subject_code and amyelg
AMYSTATUS_df = AMYSTATUS_df[['subject_code','amyelg']]
merged_Frame = pd.merge(allscans,AMYSTATUS_df, on='subject_code',how='outer')

# extract Group, Sex, Age, Date, Description
DOWNLOADS_df = DOWNLOADS_df[['Subject','Group','Sex','Age','Description','Acq Date']]
DOWNLOADS_df = DOWNLOADS_df.rename(index=str, columns={"Subject": "subject_code"}) # rename Subject to subject_code for merging
DOWNLOADS_df = DOWNLOADS_df.drop_duplicates(subset=['subject_code','Description'], keep=False)
DOWNLOADS_df['subject_code'] = DOWNLOADS_df.subject_code.astype(str).str.lower()
merged_Frame2 = pd.merge(merged_Frame,DOWNLOADS_df, on='subject_code',how='outer')

RECON3T_QC = RECON3T_QC[['LEADS_ID','OVERALLQC','TEMPQC','FRONTQC','PARQC','INSULAQC','OCCQC','BGQC','CWMQC','VENTQC','LHIPQC','RHIPQC','COMMENTSQC','FS_INITIAL_COMMAND','FS_VERSION','STATUS_FINAL','EXCLUDE','REASON_EXCLUDE','EDITOR']]
RECON3T_QC = RECON3T_QC.rename(index=str, columns={"LEADS_ID": "leads_id"})
merged_Frame3 = pd.merge(merged_Frame2,RECON3T_QC, on='leads_id',how='outer')


# NOW ADD THE STATS : add merged_Frame3 to statsmerged_df (column)
# set statsmerged_df index (to numbers)
statsmerged_df = statsmerged_df.reset_index()
statsmerged_df = statsmerged_df.rename(index=str, columns={"Measure:mean": "leads_id"})
# rename the first column to 'leads_id'
EXTERNAL_DF = pd.merge(merged_Frame3,statsmerged_df, on='leads_id',how='outer')

# save this final dataframe with the date:
EXTERNAL_DF.to_csv(externalpath+'MGH_LEADS_DATASUMMARY_3T_'+time.strftime("%Y%m%d")+'.csv', index=False)